<h1> Implementing the ReAct agent without using langgraph framework </h1>
<li>it uses "Thought - Action - Observation" pattern</li>
<li>As sample Add,Multiple methos is implemented</li>


In [106]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import re
import httpx


from langchain_groq import ChatGroq

In [107]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# _set_env("OPENAI_API_KEY")
_set_env("GROQ_API_KEY")

In [119]:
### tools

def calculate(what):
    return eval(what)

def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]




In [157]:
### Prompt Template & LLM

system_template = """You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.
Answer: the final answer to the original question

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:
# Example1
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris

# Example2
Question: What is 3 * 12 ?
Thought: I should calculate the result of 3 * 12
Action: calculate: 3 * 12
PAUSE


You will be called again with this:

Observation: 36
Answer: 36

""".strip()

human_template = """Question: {question}"""

llm = ChatGroq(model="llama3-8b-8192",temperature=0)

In [158]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
}

In [164]:
def parse_output(output):
    # print("Entered Parser - "+output)
    action_match = re.search(r'Action: (.*)', output)
    final_answer_match = re.search(r'Answer: (.*)', output)

    action = action_match.group(1).strip() if action_match else None
    final_answer = final_answer_match.group(1).strip() if final_answer_match else None

    return action, final_answer

In [196]:
### run the agent  - Agent execution is done untill the Final answer is constructed


def run_react_agent(question):
    agent_scratchpad = ""
    max_iterations = 5  # Prevent infinite loops
    messages = []
    # Initialize the messages with the system prompt
    messages.append(SystemMessage(content=system_template))
    human_message = human_template.format(question=question)
    messages.append(HumanMessage(content=human_message))

    for _ in range(max_iterations):
        print("loop in")
        response = llm.invoke(messages)
        messages.append(AIMessage(content=response.content))
        actions,final_answer = parse_output(response.content)
        # actions = [action_re.match(a) for a in response.content.split('\n') if action_re.match(a)]
        # print(actions[0].groups())
        if final_answer:
            return "Answer: {}".format(final_answer)

        if actions:
            action, action_input = actions.split(":")
            if action not in known_actions:
                raise Exception("Unknown action ") 
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            # print("Observation" , observation)
            messages.append(HumanMessage(content=f"Observation: {observation}"))
            # [msg.pretty_print() for msg in messages]
        else:
            return ""


In [197]:
query = "What is 4 * 12?"
print(run_react_agent(query))

loop in
 -- running calculate  4 * 12
loop in
Answer: 48


In [198]:
query = "What is 4 * 12 and add 3 and divide the result by 2?"
print(run_react_agent(query))

loop in
 -- running calculate  4 * 12 + 3 / 2
loop in
Answer: The result of 4 * 12 and add 3 and divide the result by 2 is 49.5.


In [180]:
query = "What is capital of India"
print(run_react_agent(query))

 -- running wikipedia  India


/tmp/ipykernel_2607/3523303771.py:2: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile('^Action: (\w+): (.*)$')


Exception: Unknown action 

In [113]:
# response = llm.invoke("What is 3 * 12?")
# print(response.additional_kwargs)
# print("----------")

# response = llm.invoke("What is multiplication 3 and 12?")
# print(response.additional_kwargs)

# print("----------")
# response = llm.invoke("addition of 3 and 12?")
# print(response.additional_kwargs)

# print("----------")
# response = llm.invoke("3*12 + 4")
# print(response.additional_kwargs)
